# test qwen2

In [1]:
# 增加norm和rope的一致性
import os
import sys
import torch
import numpy  as np
from deepspeed.profiling.flops_profiler import get_model_profile  # type: ignore
current_dir = os.getcwd()
print(current_dir)
sys.path.append(os.path.join(current_dir, "steel_modify_from_qwen_1_5"))
from transformers import AutoConfig
from steel_modify_from_qwen_1_5.modeling_steel import Qwen2ForCausalLM
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)
config.use_cuda_rmsnorm = True
config.mlp_type = "raw"
config.FFN_type = "softmoe_v1"
# ['eager', 'flash_attention_2', 'sdpa']
# config._attn_implementation = "flash_attention_2"
model = Qwen2ForCausalLM(config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
memory_allocated = torch.cuda.memory_allocated(device)
print("模型在显存中的占用大小：", memory_allocated, "字节")
for name, param in model.named_parameters():
    print(f"参数名称: {name}，数据类型: {param.dtype}")
    break
# print(model)

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-12 14:41:38,675] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible
/home/calfa100/gqs/Steel-LLM/pretrain_modify_from_TinyLlama/model
zhanshijin: surrport flash_attn_2
zhanshijin: if flash attn surrport window:True


mlp_class: SteelSENet
FFN: SteelSoftMoeV3
zhanshijin: now use _attn_implementation is sdpa, you can choose from dict_keys(['eager', 'flash_attention_2', 'sdpa'])


zhanshijin: success import cuda __rms_norm
模型在显存中的占用大小： 9238446080 字节
参数名称: model.embed_tokens.weight，数据类型: torch.float32


In [2]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): SteelSoftMoEV3(
          (experts): ModuleList(
            (0-7): 8 x SteelSENet(
              (gate_up_proj): Linear(in_features=2048, out_features=688, bias=False)
              (up_proj): Linear(in_features=2048, out_features=688, bias=False)
              (gate_down_proj): Linear(in_features=688, out_features=2048, bias=False)
              (down_proj): Linear(in_features=688, out_features=2048, bias=False)
              (act_

In [2]:

# RuntimeError: FlashAttention only support fp16 and bf16 data type
seed = 666
torch.manual_seed(seed)  # 设置随机种子
torch.cuda.manual_seed(seed)  # 设置CUDA的随机种子
torch.cuda.manual_seed_all(seed)  # 如果使用多个GPU，设置所有GPU的随机种子
np.random.seed(seed)
fake_input = torch.randint(1, 10000, (1, 256),dtype=torch.long, device=model.device)
input_dict = {"input_ids": fake_input, "labels": fake_input.clone()}
# model = model.to(torch.float32)
# flops, macs, params = get_model_profile(model, kwargs=input_dict, print_profile=True, detailed=True)
# print("FLOPs:", flops)
# print("MACs:", macs)
# print("Params:", params)
# output = model(fake_input)
# print(output.logits[:,:,:10])
# print("*"*20)
#===========================================================
model = model.to(torch.bfloat16)
flops, macs, params = get_model_profile(model, kwargs=input_dict, print_profile=True, detailed=True)
print("FLOPs:", flops)
print("MACs:", macs)
print("Params:", params)
output = model(fake_input)
print(output.logits[:,:,:10])

[2024-05-12 03:36:42,547] [INFO] [profiler.py:1218:get_model_profile] Flops profiler warming-up...


[2024-05-12 03:36:43,142] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 1:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         1.84 B  
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       189.99 GMACs
fwd flops per GPU:                                                      380.59 G
fwd flops of model = fwd flops per GPU * mp_size:                     

# 测试 rms+rope一致性

In [3]:
from flash_attn.layers.rotary import apply_rotary_emb_func as __apply_rotary_emb_func
from flash_attn.ops.rms_norm import rms_norm as __rms_norm
from torch import nn
import torch
class Qwen2RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        Qwen2RMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

In [12]:
eps = 1e-6
dim = 16
input  = torch.randn([1,dim],device="cuda:0",dtype=torch.bfloat16)
torch_rmsnorm = Qwen2RMSNorm(dim, eps=eps).to("cuda:0")
print(torch_rmsnorm.weight.dtype)
output1 = torch_rmsnorm(input)
#=
output2 = __rms_norm(input, torch_rmsnorm.weight, torch_rmsnorm.variance_epsilon)
print(output1)
print(output2)

torch.float32
tensor([[ 0.3965, -0.6992, -1.8359, -0.8242, -1.3672,  1.4688,  0.1836,  1.0703,
         -1.0859, -1.9062, -0.2969, -0.3633, -0.9648, -0.0815,  0.1748, -0.3105]],
       device='cuda:0', grad_fn=<MulBackward0>)
tensor([[ 0.3965, -0.6992, -1.8359, -0.8242, -1.3672,  1.4688,  0.1836,  1.0703,
         -1.0859, -1.9062, -0.2969, -0.3633, -0.9648, -0.0815,  0.1748, -0.3105]],
       device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<DropoutAddLayerNormFnBackward>)


In [ ]:
# test recurrentgemma
from transformers import AutoConfig
from recurrentgemma.modeling_recurrent_gemma import RecurrentGemmaForCausalLM
config = AutoConfig.from_pretrained("./recurrentgemma",trust_remote_code=True)
model = RecurrentGemmaForCausalLM(config)
print(model)

# load model

In [4]:
import os
import sys
import torch
import numpy  as np
import lightning as L
from deepspeed.profiling.flops_profiler import get_model_profile  # type: ignore
fabric = L.Fabric(devices=1, precision="bf16-mixed")
current_dir = os.getcwd()
print(current_dir)
sys.path.append(os.path.join(current_dir, "steel_modify_from_qwen_1_5"))
from transformers import AutoConfig
from steel_modify_from_qwen_1_5.modeling_steel import Qwen2ForCausalLM
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)
config.FFN_type = "raw"
config.mlp_div_ratio = 1
ckpt = "/home/calfa100/gqs/Steel-LLM/pretrain_modify_from_TinyLlama/pretrain/out/qwen1_5_step-030000-iter-120000-ckpt/state.pth"
model = Qwen2ForCausalLM(config).to("cuda:0").to(torch.bfloat16)
state = {"model": model}
fabric.load(ckpt, state)


Using bfloat16 Automatic Mixed Precision (AMP)


/home/calfa100/gqs/Steel-LLM/pretrain_modify_from_TinyLlama/model
zhanshijin: use torch rmsnorm


{'optimizer': {'state': {'lm_head.weight': {'step': tensor(30000.),
    'exp_avg': tensor([[-1.0488e-06, -7.1102e-06,  1.1775e-06,  ...,  4.8751e-06,
              7.7625e-07, -1.2856e-06],
            [ 3.5733e-07,  2.1447e-06,  1.1023e-05,  ...,  2.0839e-06,
             -3.6291e-06,  3.7009e-06],
            [-1.6473e-06, -1.7232e-06, -5.8710e-07,  ...,  8.9671e-07,
              1.4101e-06, -1.6188e-06],
            ...,
            [-1.2707e-10, -3.7696e-10,  1.0281e-10,  ..., -8.1038e-11,
             -4.5841e-11,  6.3461e-10],
            [-1.1979e-10, -3.7911e-10,  8.7173e-11,  ..., -8.9802e-11,
             -4.4945e-11,  6.3324e-10],
            [-1.2511e-10, -3.7976e-10,  1.0645e-10,  ..., -9.3724e-11,
             -3.7553e-11,  6.2820e-10]]),
    'exp_avg_sq': tensor([[3.0918e-10, 2.2391e-10, 1.7112e-10,  ..., 1.8098e-10, 1.7663e-10,
             2.9262e-10],
            [4.9223e-10, 4.5523e-10, 7.3779e-10,  ..., 4.0494e-10, 3.7681e-10,
             3.0057e-10],
            

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/calfa100/gqs/Steel-LLM/pretrain_modify_from_TinyLlama/model/tokenizer_from_qwen_moe_chat")
model_inputs = tokenizer(["我好看吗"], return_tensors="pt").to("cuda:0")
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=100
)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


我好看吗？我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你，我好喜欢你


# softmoe v1

In [10]:
from torch import nn
import torch.nn.functional as F
import torch
from transformers import AutoConfig
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)

class SteelSoftMoeV1(nn.Module):
    def __init__(self, config, layer=None):
        super().__init__()
        self.config = config
        # self.experts = nn.ModuleList([layer(config) for _ in range(config.n_experts)])
        self.experts = nn.ModuleList([nn.Linear(config.hidden_size, 10) for _ in range(config.n_experts)])
        self.gating = nn.Linear(config.hidden_size, config.n_experts)
    def forward(self, x):
        weights = self.gating(x)
        weights = nn.functional.softmax(weights, dim=-1, dtype=torch.float32).to(x.dtype)
        outputs = torch.stack( 
            [expert(x) for expert in self.experts], dim=2) 
        weights = weights.unsqueeze(-1)
        return torch.sum(outputs * weights, dim=2)
    
fake_input = torch.randn([1,30,config.hidden_size])
layer = SteelSoftMoeV1(config)
print(layer(fake_input).shape)

torch.Size([1, 30, 8])
torch.Size([1, 30, 8, 10])
torch.Size([1, 30, 10])
